### Extra en Espanol Transcript Extraction

In [3]:
!pip install textract

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.3.0 requires gast==0.3.3, but you have gast 0.4.0 which is incompatible.
tensorflow 2.3.0 requires numpy<1.19.0,>=1.16.0, but you have numpy 1.20.1+mkl which is incompatible.
tensorflow 2.3.0 requires opt-einsum>=2.3.2, but you have opt-einsum 0+untagged.61.gd905544.dirty which is incompatible.
tensorflow 2.3.0 requires scipy==1.4.1, but you have scipy 1.6.0 which is incompatible.
tensorflow 2.3.0 requires tensorflow-estimator<2.4.0,>=2.3.0, but you have tensorflow-estimator 2.4.0 which is incompatible.
pipwin 0.5.1 requires beautifulsoup4>=4.9.0, but you have beautifulsoup4 4.8.0 which is incompatible.
google-api-core 1.25.1 requires six>=1.13.0, but you have six 1.12.0 which is incompatible.



  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3966 sha256=dcd03dd1115dc2654621b2bf03db824b2e51d27783a94c4be992d0b5455a9436
  Stored in directory: c:\users\dasha\appdata\local\pip\cache\wheels\55\f0\2c\81637d42670985178b77df6d41b9b6c6dc18c94818447414b9
  Created wheel for EbookLib: filename=EbookLib-0.17.1-py3-none-any.whl size=38162 sha256=32ef553600f7ec2f6236eba2c0c502dd95ef33efe0b340080b1b5eb607834db0
  Stored in directory: c:\users\dasha\appdata\local\pip\cache\wheels\b4\eb\66\00c65b5bbf31ec34329090ec9fe8c8a8d9cb7a3a3d93841386
  Created wheel for python-pptx: filename=python_pptx-0.6.18-py3-none-any.whl size=275704 sha256=b695aeb3cf9f96dec656e088e9bffc605149b924f648c76dff30ee9890c49e9a
  Stored in directory: c:\users\dasha\appdata\local\pip\cache\wheels\11\2b\97\d82ca57932fa62d52c723024419c5ec3b7c0f7ecf0a0f06332
  Created wheel for tzlocal: filename=tzlocal-1.5.1-py3-none-any.whl size=17541 sha256=f7ca2d885ec765e77b56e54d5328dd64bbade180c0669174cabd891c8

In [4]:
import textract

In [14]:
#files_dir = "/Capstone/Extra en Espanol/"
files_dir = "C:/Users/dasha/Documents/Education/UBC/Capstone/Extra en Espanol/"
filenames = ['01 La llegada de Sam.docx',
              '02 Sam va de compras.doc',
              '03 Sam aprende a ligar.doc',
              '04 Sam busca un trabajo.doc',
              '05 Ha nacido una estrella.docx',
              '06 El día de la primitiva.doc',
              '07 La gemela.doc',
              '08 La prima de la dueña.doc',
              '09 Trabajos para los chicos.doc',
              '10 ANA PROTESTA.doc',
              '11 Tiempo de vacaciones.doc',
              '12 Fanáticos del fútbol.doc',
              '13 Boda en el aire.doc']

In [25]:
text_dir = 'C:/Users/dasha/Documents/Education/UBC/Capstone/capstone_FHIS/text/'
text_list = []
for filename in filenames:
    text = textract.process(files_dir+filename).decode('utf-8')
    with open(text_dir + filename[:filename.index('.')] + ".txt", 'w') as text_file:
        text_file.write(text)
    text_list.append(text)

In [ ]:
# Create JSON files
from collections import defaultdict
dict_list = []
author = "Channel 4 Learning" #the author is the producer in this case
level = "A1"
source_list = ['https://www.dropbox.com/s/8ia5r3wfsmobg07/01%20La%20llegada%20de%20Sam.docx?dl=0',
              'https://www.dropbox.com/s/x92adnk4cp3u93p/02%20Sam%20va%20de%20compras.doc',
              '']
for idx, text in enumerate(text_list):
    # metadata
    # source
    source = source_list[idx]
    
    # Title
    title_start = filenames[idx].index(' ')+1
    title_end = filenames[idx].index('.')
    title = filenames[idx][title_start:title_end]
    
    # content
    content = text
    # make dictionary
    text_dict = defaultdict(str)
    text_dict['author'] = author
    text_dict['source'] = source
    text_dict['level'] = level
    text_dict['title'] = title
    text_dict['content'] = content
    dict_list.append(text_dict)

In [ ]:
import json
json_dir = "C:/Users/dasha/Documents/Education/UBC/Capstone/capstone_FHIS/corpus/"
with open(files_dir + 'extra_en_espanol.json', 'w') as outfile:
    json.dump(dict_list, outfile)